In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pdb
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import pywt

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, RepeatVector, TimeDistributed, Flatten, LSTM, Bidirectional, Dropout, BatchNormalization, Conv2D, Activation, MaxPooling2D, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import mse
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import glorot_normal
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import librosa
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
import time
import os
import shutil
from tqdm import tqdm
mm = MinMaxScaler()

In [ ]:
import pickle
# load PPG data
file_open=open('xxx/UBFC_data/cycle_ppg_UBFC_T1_S1_S56.pl','rb')
user_data_x,user_data_y=pickle.load(file_open)
file_open.close()

# load rPPG data
file_open=open('xxx/UBFC_data/cycle_rppg_UBFC_T1_S1_S56.pl','rb')
user_data_rppgx,user_data_rppgy=pickle.load(file_open)
file_open.close()

y_list = list(Counter(user_data_rppgy).keys()) 

## CNN-LSTM

In [ ]:
arr_result = []

for sub_number in y_list:
  arr_sub_arr_result=[]
  test_ID = sub_number
  print(test_ID)
  arr_size = 60
  user_test_data_arr = []
  cycle_ppg_arr_x = []
  cycle_ppg_arr_y = []
  train_data_x = []
  test_data_x = []
  for index, data in enumerate(user_data_x) :
    if user_data_y[index]!=test_ID:
      continue
    resample = scipy.signal.resample(data,arr_size)
    resample_nor  =  mm.fit_transform(resample.reshape(-1,1)).flatten()
    user_test_data_arr.append(resample_nor)
  mean_arr = np.mean(user_test_data_arr, axis=0) 
  for i in user_test_data_arr:
    if stats.pearsonr(i,mean_arr)[0]<0.9:
      continue
    else:
      cycle_ppg_arr_x.append(i)
      cycle_ppg_arr_y.append(test_ID)
      train_data_x.append(i)

  for index, data in enumerate(user_data_x) :

    if user_data_y[index]==test_ID:
      continue
    resample = scipy.signal.resample(data, arr_size)
    resample_nor  = mm.fit_transform(resample.reshape(-1,1)).flatten()

    if index%10==0:
      cycle_ppg_arr_x.append(resample_nor)
      cycle_ppg_arr_y.append(user_data_y[index])
    else:
      test_data_x.append(resample_nor)
  
  

  X = np.array(cycle_ppg_arr_x)
  Y = np.array(cycle_ppg_arr_y)

  # print('test ID',test_ID)
  Y[np.where(Y!=test_ID)] = 0

  arr_sub_arr_result.append(test_ID)

  # number of class
  n_classes= 2
  # number of features
  n_features=X.shape[1]

  enc = OneHotEncoder()
  # randomly shuffle data before training and testing

  randIndx = np.arange(X.shape[0])
  np.random.seed(111)
  np.random.shuffle(randIndx)

  trainSamples=np.floor(X.shape[0]*0.7).astype(int)
  testSamples=np.floor(X.shape[0]*0.3).astype(int) 


  X_new = X[randIndx,:] 
  
  Y_new =enc.fit_transform(Y[randIndx].reshape(-1,1)).toarray() 


  # Assign training and testing set
  nChannels=1

  train_data, train_target = np.reshape(X_new[:trainSamples,:],[trainSamples,n_features,nChannels]), Y_new[:trainSamples,:]
  try:
    test_data, test_target = np.reshape(X_new[trainSamples+1:,:],[testSamples,n_features,nChannels]), Y_new[trainSamples+1:,:]
  except:
    test_data, test_target = np.reshape(X_new[trainSamples:,:],[testSamples,n_features,nChannels]), Y_new[trainSamples:,:]



  unique, counts = np.unique(train_target[:,1], return_counts=True)
  arr_sub_arr_result.append(counts[0])
  arr_sub_arr_result.append(counts[1])

  unique, counts = np.unique(test_target[:,1], return_counts=True)
  arr_sub_arr_result.append(counts[0])
  arr_sub_arr_result.append(counts[1])

  #first CNN
  model = Sequential()
  model.add(Conv1D(filters=25, kernel_size=6, padding='valid',strides=1, input_shape=[60,1]))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=4))
  model.add(Dropout(0.25))

  #second CNN
  model.add(Conv1D(filters=40, kernel_size=10, padding='valid'))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=4))
  model.add(Dropout(0.25))

  #first LSTM. note that we need to do a timedistributed flatten as a transition from CNN to LSTM
  model.add(TimeDistributed(Flatten()))
  model.add(Bidirectional(LSTM(units=32, return_sequences=True, dropout=0.25)))
  model.add(Flatten())

  #activation layer
  model.add(Dense(2, activation='softmax'))

  #compile model
  model.compile(
      loss='categorical_crossentropy',
      # loss='binary_crossentropy',

      optimizer=Adam(),
      metrics=['accuracy'],
  )

  #LSTM parameters
  BATCH_SIZE =20 #batch size
  EPOCHS = 50     #number of epochs


  # early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)

  fited_model = model.fit(
    train_data,
    train_target,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    # callbacks=[early_stop],
    shuffle=True,
    validation_data=(test_data,test_target),
  )

  model.save(save_model_location+'/model_'+str(test_ID)+'.h5')



  # Find Threshold
  thres = np.arange(0, 1.0001, 0.0001)

  thres_matrix_test = np.zeros((len(test_target),len(thres)))
  total_matrix_test = np.zeros((len(thres),4)) #acc,far,frr,recall(order of column)
  test_logit_temp=model.predict(test_data)  

  for j in range(len(thres)):
      for i in range(len(test_target)):
          if test_logit_temp[i][1] < thres[j]:
              thres_matrix_test[i,j] = 0
          else:
              thres_matrix_test[i,j] = 1

  for i in range(len(thres)):

      equal_logit = np.equal(thres_matrix_test[:,i], test_target[:,1]) ##Test
      unique2, True_pos_neg_test = np.unique(thres_matrix_test[np.where(equal_logit==True),i], return_counts=True)
      unique3, False_pos_neg_test = np.unique(thres_matrix_test[np.where(equal_logit==False),i], return_counts=True) 

      if np.shape(True_pos_neg_test)==(1,):
          if unique2[0] == 0:
              True_pos_neg_test = [True_pos_neg_test[0], 0]
          else:
              True_pos_neg_test = [0, True_pos_neg_test[0]]

      if np.shape(False_pos_neg_test)==(1,):
          if unique3[0] == 0:
              False_pos_neg_test = [False_pos_neg_test[0], 0]
          else:
              False_pos_neg_test = [0, False_pos_neg_test[0]]
    
      if np.shape(True_pos_neg_test)==(0,):
          True_pos_neg_test = [0, 0]

      if np.shape(False_pos_neg_test)==(0,):
          False_pos_neg_test = [0, 0]
  
      Recall_test= True_pos_neg_test[1]/(False_pos_neg_test[0]+True_pos_neg_test[1])
      Specific_test = True_pos_neg_test[0] /(True_pos_neg_test[0]+False_pos_neg_test[1])

      ACC_test = (True_pos_neg_test[0]+True_pos_neg_test[1]) / (len(test_target))
      FAR_test = 1 - Specific_test
      FRR_test = 1 - Recall_test

      total_matrix_test[i,:] = ACC_test, FAR_test, FRR_test, Recall_test

  fig4 = plt.figure()
  ax = plt.subplot(111)
  ax.plot(thres, total_matrix_test[:,1], 'b', label='FAR')
  ax.plot(thres, total_matrix_test[:,2],'r', label='FRR')
  ax.legend()
  plt.title('FAR vs FRR (Test)')
  plt.xlabel('Threshold')
  plt.ylabel('Err Rate')
  plt.show()

  EER_line = thres[::-1]
  fig5 = plt.figure()
  ax2 = plt.subplot(111)
  ax2.plot(total_matrix_test[:,1], total_matrix_test[:,3],'r',label='ROC')
  ax2.plot(thres, EER_line, 'b',label='EER Line')
  ax2.legend()
  plt.title('ROC Curve (Test)')
  plt.xlabel('FAR')
  plt.ylabel('TPR')
  plt.show()

  EER_loc_test = np.argmin(abs(total_matrix_test[:,1] - total_matrix_test[:,2]))
  # print('EER threshold: ', thres[EER_loc_test])
  print('EER: {:.3f} '.format((total_matrix_test[EER_loc_test,1]+total_matrix_test[EER_loc_test,2])/2))
  print('FRR at EER: {:.3f} '.format(total_matrix_test[EER_loc_test,2]))
  print('FAR at EER: {:.3f} '.format(total_matrix_test[EER_loc_test,1]))
  print('Accuracy at EER: {:.3f} '.format(total_matrix_test[EER_loc_test,0]))


  file_open=open('xxx/UBFC_data/CNN_LSTM_model/thres_EER_loc_test_'+str(test_ID)+'.pl','wb')
  pickle.dump((thres,EER_loc_test),file_open)
  file_open.close()

## Evaluation

In [ ]:
# y_list = list(Counter(user_data_rppgy).keys()) 
arr_result = []

for sub_number in y_list:
  # if sub_number not in y_list1:
  #   continue

  arr_sub_arr_result=[]
  test_ID = sub_number
  print(test_ID)
  arr_size = 60
  user_test_data_arr = []
  cycle_ppg_arr_x = []
  cycle_ppg_arr_y = []
  train_data_x = []
  test_data_x = []
  for index, data in enumerate(user_data_x) :
    if user_data_y[index]!=test_ID:
      continue
    resample = scipy.signal.resample(data,arr_size)
    resample_nor  =  mm.fit_transform(resample.reshape(-1,1)).flatten()
    user_test_data_arr.append(resample_nor)
  mean_arr = np.mean(user_test_data_arr, axis=0) 
  for i in user_test_data_arr:
    if stats.pearsonr(i,mean_arr)[0]<0.9:
      continue
    else:
      cycle_ppg_arr_x.append(i)
      cycle_ppg_arr_y.append(test_ID)
      train_data_x.append(i)

  for index, data in enumerate(user_data_x) :

    if user_data_y[index]==test_ID:
      continue
    resample = scipy.signal.resample(data, arr_size)
    resample_nor  = mm.fit_transform(resample.reshape(-1,1)).flatten()

    if index%10==0:
      cycle_ppg_arr_x.append(resample_nor)
      cycle_ppg_arr_y.append(user_data_y[index])
    else:
      test_data_x.append(resample_nor)
  
  

  X = np.array(cycle_ppg_arr_x)
  Y = np.array(cycle_ppg_arr_y)

  # print('test ID',test_ID)
  Y[np.where(Y!=test_ID)] = 0

  arr_sub_arr_result.append(test_ID)

  # number of class
  n_classes= 2
  # number of features
  n_features=X.shape[1]

  enc = OneHotEncoder()
  # randomly shuffle data before training and testing

  randIndx = np.arange(X.shape[0])
  np.random.seed(111)
  np.random.shuffle(randIndx)

  trainSamples=np.floor(X.shape[0]*0.7).astype(int)
  testSamples=np.floor(X.shape[0]*0.3).astype(int) 


  X_new = X[randIndx,:] 
  
  Y_new =enc.fit_transform(Y[randIndx].reshape(-1,1)).toarray() 


  # Assign training and testing set
  nChannels=1

  train_data, train_target = np.reshape(X_new[:trainSamples,:],[trainSamples,n_features,nChannels]), Y_new[:trainSamples,:]
  try:
    test_data, test_target = np.reshape(X_new[trainSamples+1:,:],[testSamples,n_features,nChannels]), Y_new[trainSamples+1:,:]
  except:
    test_data, test_target = np.reshape(X_new[trainSamples:,:],[testSamples,n_features,nChannels]), Y_new[trainSamples:,:]


  # print("train:",train_data.shape,train_target.shape)
  # print("test:",test_data.shape,test_target.shape)

  # Check # of 1 and 0 in training and testing set

  unique, counts = np.unique(train_target[:,1], return_counts=True)
  # print('Train set dictionary: ',dict(zip(unique, counts)))
  arr_sub_arr_result.append(counts[0])
  arr_sub_arr_result.append(counts[1])

  unique, counts = np.unique(test_target[:,1], return_counts=True)
  # print('Valid set dictionary: ',dict(zip(unique, counts)))
  arr_sub_arr_result.append(counts[0])
  arr_sub_arr_result.append(counts[1])

  try:
    model = load_model(save_model_location+'/model_'+str(test_ID)+'.h5')
  except:
    continue
  thres = np.arange(0, 1.0001, 0.0001)

  thres_matrix_test = np.zeros((len(test_target),len(thres)))
  total_matrix_test = np.zeros((len(thres),4)) #acc,far,frr,recall(order of column)
  test_logit_temp=model.predict(test_data)  

  for j in range(len(thres)):
      for i in range(len(test_target)):
          if test_logit_temp[i][1] < thres[j]:
              thres_matrix_test[i,j] = 0
          else:
              thres_matrix_test[i,j] = 1

  for i in range(len(thres)):

      equal_logit = np.equal(thres_matrix_test[:,i], test_target[:,1]) ##Test
      unique2, True_pos_neg_test = np.unique(thres_matrix_test[np.where(equal_logit==True),i], return_counts=True)
      unique3, False_pos_neg_test = np.unique(thres_matrix_test[np.where(equal_logit==False),i], return_counts=True) 

      if np.shape(True_pos_neg_test)==(1,):
          if unique2[0] == 0:
              True_pos_neg_test = [True_pos_neg_test[0], 0]
          else:
              True_pos_neg_test = [0, True_pos_neg_test[0]]

      if np.shape(False_pos_neg_test)==(1,):
          if unique3[0] == 0:
              False_pos_neg_test = [False_pos_neg_test[0], 0]
          else:
              False_pos_neg_test = [0, False_pos_neg_test[0]]
    
      if np.shape(True_pos_neg_test)==(0,):
          True_pos_neg_test = [0, 0]
          
      if np.shape(False_pos_neg_test)==(0,):
          False_pos_neg_test = [0, 0]
  
      Recall_test= True_pos_neg_test[1]/(False_pos_neg_test[0]+True_pos_neg_test[1])
      Specific_test = True_pos_neg_test[0] /(True_pos_neg_test[0]+False_pos_neg_test[1])

      ACC_test = (True_pos_neg_test[0]+True_pos_neg_test[1]) / (len(test_target))
      FAR_test = 1 - Specific_test
      FRR_test = 1 - Recall_test

      total_matrix_test[i,:] = ACC_test, FAR_test, FRR_test, Recall_test


  EER_loc_test = np.argmin(abs(total_matrix_test[:,1] - total_matrix_test[:,2]))

  arr_sub_arr_result.append((total_matrix_test[EER_loc_test,1]+total_matrix_test[EER_loc_test,2])/2)
  arr_sub_arr_result.append(total_matrix_test[EER_loc_test,2])
  arr_sub_arr_result.append(total_matrix_test[EER_loc_test,1])
  arr_sub_arr_result.append(total_matrix_test[EER_loc_test,0])

  test_data_x_pre =np.expand_dims(test_data_x,2) 

  result = model.predict(test_data_x_pre)
  final_result = result[:,1]
  final_result[np.where(final_result>thres[EER_loc_test])] = 1
  final_result[np.where(final_result<thres[EER_loc_test])] = 0

  true_y = np.zeros(len(final_result))
  # print("Test data number:",len(true_y))
  # print("FAR:",(final_result[final_result == 1 ].size)/len(final_result))
  
  arr_sub_arr_result.append(len(true_y))
  arr_sub_arr_result.append((final_result[final_result == 1 ].size)/len(final_result))

  try:
    # replace WGAN GP GMM result here #################################################
    file_open=open('wgan_rppg_result_'+str(test_ID)+'.pl','rb')
  except:
    print('except')
    continue
  b, preds_b=pickle.load(file_open)
  file_open.close()


  result = model.predict(np.expand_dims(b,2))
  final_result = result[:,1]
  final_result[np.where(final_result>thres[EER_loc_test])] = 1
  final_result[np.where(final_result<thres[EER_loc_test])] = 0
  true_y = np.zeros(len(final_result))
  # print("rPPG data number:",len(true_y))
  # print("rPPG FAR:",(final_result[final_result == 1 ].size)/len(final_result))
  arr_sub_arr_result.append(len(true_y))
  arr_sub_arr_result.append((final_result[final_result == 1 ].size)/len(final_result))

  preds_b = np.squeeze(preds_b)
  result = model.predict(np.expand_dims(preds_b,2))
  final_result = result[:,1]
  final_result[np.where(final_result>thres[EER_loc_test])] = 1
  final_result[np.where(final_result<thres[EER_loc_test])] = 0
  true_y = np.zeros(len(final_result))
  (final_result[final_result == 1 ].size)/len(final_result)
  arr_sub_arr_result.append((final_result[final_result == 1 ].size)/len(final_result))


  sub_number=2
  max_pre = arr_sub_arr_result[-2]
  max_number = 0
  while sub_number < len(b):

    mean_add = []
    sub_data = []
    for index, i in  enumerate(b):
      if index%sub_number==0 and index!=0:
        mean_add.append(np.mean(sub_data, axis=0) ) 
        sub_data=[]
      else:
        resample = scipy.signal.resample(i,arr_size)
        resample_nor  =  mm.fit_transform(resample.reshape(-1,1)).flatten()
        sub_data.append(resample_nor)
    sub_number+=1
    mean_add = np.expand_dims(mean_add,2)
    result = model.predict(mean_add)
    final_result = result[:,1]
    final_result[np.where(final_result>thres[EER_loc_test])] = 1
    final_result[np.where(final_result<thres[EER_loc_test])] = 0
    true_y = np.zeros(len(final_result))
    far_result = (final_result[final_result == 1 ].size)/len(final_result)
    if far_result > max_pre:
      max_pre = far_result
      max_number = sub_number-1
    if max_pre==1:
      break
  # print(max_pre)
  # print(max_number)

  arr_sub_arr_result.append(max_pre)
  arr_sub_arr_result.append(max_number)


  sub_number=2
  max_pre = arr_sub_arr_result[-3]
  max_number = 0
  last_len_data = 0
  while sub_number < len(preds_b):

    mean_add = []
    sub_data = []
    for index, i in  enumerate(preds_b):
      if index%sub_number==0 and index!=0:
        mean_add.append(np.mean(sub_data, axis=0) ) 
        sub_data=[]
      else:
        resample = scipy.signal.resample(i,arr_size)
        resample_nor  =  mm.fit_transform(resample.reshape(-1,1)).flatten()
        sub_data.append(resample_nor)
    if last_len_data == len(mean_add):
      sub_number+=1
      last_len_data=len(mean_add)
      continue
    last_len_data=len(mean_add)
    mean_add = np.expand_dims(mean_add,2)
    result = model.predict(mean_add)
    final_result = result[:,1]
    final_result[np.where(final_result>thres[EER_loc_test])] = 1
    final_result[np.where(final_result<thres[EER_loc_test])] = 0
    true_y = np.zeros(len(final_result))
    far_result = (final_result[final_result == 1 ].size)/len(final_result)
    if far_result > max_pre:
      max_pre = far_result
      max_number = sub_number-1
    if max_pre==1:
      break

  arr_sub_arr_result.append(max_pre)
  arr_sub_arr_result.append(max_number)
  arr_result.append(arr_sub_arr_result)



In [ ]:
# print result
import pandas as pd
df2 = pd.DataFrame(arr_result,
                   columns=['Test ID', 'Train0', 'Train1','Test0', 'Test1','EER','FRR','FAR', 'ACC','Test data',' Test FAR','rPPG number','rPPG FAR','WGAN FAR','rPPG MAX FAR','rPPG MAX num','WGAN MAX FAR', 'WGAN MAX num'])

print('acc:',df2['ACC'].mean())
print('FAR:',df2['FAR'].mean())
print('rPPG FAR:',df2['rPPG FAR'].mean())
print('WGAN FAR:',df2['WGAN FAR'].mean())
print('rPPG MAX FAR:',df2['rPPG MAX FAR'].mean())
print('WGAN MAX FAR:',df2['WGAN MAX FAR'].mean())


